In [262]:
from os import environ as ENV
from dotenv import load_dotenv
import datetime

import streamlit as st
import pyodbc
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [263]:
def get_db_connection() -> pyodbc.Connection:
    """Returns a connection to database."""

    load_dotenv()
    conn_str = (f"DRIVER={{{ENV['DB_DRIVER']}}};SERVER={ENV['DB_HOST']};"
                f"PORT={ENV['DB_PORT']};DATABASE={ENV['DB_NAME']};"
                f"UID={ENV['DB_USER']};PWD={ENV['DB_PASSWORD']};Encrypt=no;")

    conn = pyodbc.connect(conn_str)

    return conn


def load_data() -> pd.DataFrame:
    """ Pulls all the transaction data from the S3 bucket. """
    conn = get_db_connection()

    query = """
    SELECT
        pr.*, p.*, c.*
    FROM
        gamma.plant_reading pr
    LEFT JOIN
        gamma.plant p
    ON pr.plant_id = p.plant_id
    LEFT JOIN
        gamma.city c
    ON p.city_id = c.city_id;
    """

    df_recordings = pd.read_sql(query, conn)
    df_recordings.set_index("reading_id")
    return df_recordings

In [264]:
load_dotenv()
data = load_data()
data

/var/folders/yx/2vvnk19j32g5_ydz9hxjpgxc0000gp/T/ipykernel_57997/1118687527.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_recordings = pd.read_sql(query, conn)


,reading_id,plant_id,botanist_id,temperature,last_watered,soil_moisture,recording_taken,plant_id,plant_name,lat,lang,city_id,scientific_name,city_id,city_name,state_name
0,1,1,1,14,2025-09-25 13:51:41,32.97,2025-09-26 09:28:06.098,1,Venus flytrap,43.74,-11.51,1,None,1,Stammside,Albania
1,2,2,2,14,2025-09-25 14:58:34,34.15,2025-09-26 09:28:04.689,2,Corpse flower,47.84,-48.71,2,None,2,Floshire,American Samoa
2,3,3,3,80,2025-09-25 13:58:19,35.61,2025-09-26 09:28:05.186,3,Rafflesia arnoldii,-25.49,-36.13,3,None,3,Dale City,Mozambique
3,4,4,4,16,2025-09-25 14:56:07,36.58,2025-09-26 09:28:06.053,4,Black bat flower,63.37,46.40,4,None,4,West Tedboro,Taiwan
4,5,5,5,16,2025-09-25 13:57:08,29.81,2025-09-26 09:28:05.664,5,Pitcher plant,82.89,0.63,5,'Sarracenia catesbaei',5,North Felicia,Saint Kitts and Nevis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10358,10359,41,42,95,2025-09-26 13:08:03,99.76,2025-09-26 13:12:04.606,41,Medinilla Magnifica,-34.19,-78.70,42,'Medinilla magnifica',42,Kathrynville,Macao
10359,10360,42,43,12,2025-09-25 14:50:57,-14.96,2025-09-26 13:12:05.620,42,Calliandra Haematocephala,83.92,-148.55,43,'Calliandra haematocephala',43,Gavinstad,Jersey
10360,10361,43,44,15,2025-09-25 14:09:29,18.43,2025-09-26 13:12:04.845,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland
10361,10362,47,48,17,2025-09-25 14:08:03,20.23,2025-09-26 13:12:05.716,47,Crassula Ovata,15.80,148.99,48,'Crassula ovata',48,Ginaberg,Togo


In [265]:
last_watered_df = data[['last_watered', 'plant_name', 'soil_moisture', 'temperature']]
last_watered_df

,last_watered,plant_name,soil_moisture,temperature
0,2025-09-25 13:51:41,Venus flytrap,32.97,14
1,2025-09-25 14:58:34,Corpse flower,34.15,14
2,2025-09-25 13:58:19,Rafflesia arnoldii,35.61,80
3,2025-09-25 14:56:07,Black bat flower,36.58,16
4,2025-09-25 13:57:08,Pitcher plant,29.81,16
...,...,...,...,...
10358,2025-09-26 13:08:03,Medinilla Magnifica,99.76,95
10359,2025-09-25 14:50:57,Calliandra Haematocephala,-14.96,12
10360,2025-09-25 14:09:29,Zamioculcas Zamiifolia,18.43,15
10361,2025-09-25 14:08:03,Crassula Ovata,20.23,17


In [266]:
data.sort_values(by=['plant_name', 'last_watered'])

,reading_id,plant_id,botanist_id,temperature,last_watered,soil_moisture,recording_taken,plant_id,plant_name,lat,lang,city_id,scientific_name,city_id,city_name,state_name
31,32,32,33,12,2025-09-25 14:18:52,32.53,2025-09-26 09:28:04.944,32,Aglaonema Commutatum,-59.67,22.17,33,'Aglaonema commutatum',33,New Danika,Western Sahara
75,76,32,33,12,2025-09-25 14:18:52,35.14,2025-09-26 09:29:04.364,32,Aglaonema Commutatum,-59.67,22.17,33,'Aglaonema commutatum',33,New Danika,Western Sahara
122,123,32,33,12,2025-09-25 14:18:52,32.08,2025-09-26 09:30:05.028,32,Aglaonema Commutatum,-59.67,22.17,33,'Aglaonema commutatum',33,New Danika,Western Sahara
168,169,32,33,12,2025-09-25 14:18:52,35.15,2025-09-26 09:31:05.359,32,Aglaonema Commutatum,-59.67,22.17,33,'Aglaonema commutatum',33,New Danika,Western Sahara
214,215,32,33,12,2025-09-25 14:18:52,33.72,2025-09-26 09:32:04.767,32,Aglaonema Commutatum,-59.67,22.17,33,'Aglaonema commutatum',33,New Danika,Western Sahara
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10130,10131,43,44,15,2025-09-25 14:09:29,17.51,2025-09-26 13:07:05.392,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland
10222,10223,43,44,19,2025-09-25 14:09:29,18.93,2025-09-26 13:09:04.691,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland
10269,10270,43,44,15,2025-09-25 14:09:29,23.44,2025-09-26 13:10:05.581,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland
10315,10316,43,44,15,2025-09-25 14:09:29,19.36,2025-09-26 13:11:04.913,43,Zamioculcas Zamiifolia,-82.18,-72.49,44,'Zamioculcas zamiifolia',44,Lake Norbertstead,Greenland


In [267]:
last_watered_df = last_watered_df.drop_duplicates()
last_watered_df

,last_watered,plant_name,soil_moisture,temperature
0,2025-09-25 13:51:41,Venus flytrap,32.97,14
1,2025-09-25 14:58:34,Corpse flower,34.15,14
2,2025-09-25 13:58:19,Rafflesia arnoldii,35.61,80
3,2025-09-25 14:56:07,Black bat flower,36.58,16
4,2025-09-25 13:57:08,Pitcher plant,29.81,16
...,...,...,...,...
10358,2025-09-26 13:08:03,Medinilla Magnifica,99.76,95
10359,2025-09-25 14:50:57,Calliandra Haematocephala,-14.96,12
10360,2025-09-25 14:09:29,Zamioculcas Zamiifolia,18.43,15
10361,2025-09-25 14:08:03,Crassula Ovata,20.23,17


In [268]:
def format_timedelta(delta: datetime.timedelta) -> str:
    """ Formats timedelta into a more readable format. """
    seconds_in_day = 86400
    seconds_in_hour = 3600
    seconds_in_minute = 60

    seconds = int(delta.total_seconds())

    days = seconds // seconds_in_day
    hours = (seconds % seconds_in_day) // seconds_in_hour
    minutes = (seconds % seconds_in_hour) // seconds_in_minute

    return f'{days} days, {hours} hours, {minutes} minutes'

In [269]:
now = datetime.datetime.now()
last_watered_df['hours_since_watered'] = (now - last_watered_df['last_watered']).dt.total_seconds() / 3600
last_watered_df['time_since_watered_formatted'] = (
    now - last_watered_df['last_watered']).apply(format_timedelta)
last_watered_df

/var/folders/yx/2vvnk19j32g5_ydz9hxjpgxc0000gp/T/ipykernel_57997/3715007286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_watered_df['hours_since_watered'] = (now - last_watered_df['last_watered']).dt.total_seconds() / 3600
/var/folders/yx/2vvnk19j32g5_ydz9hxjpgxc0000gp/T/ipykernel_57997/3715007286.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_watered_df['time_since_watered_formatted'] = (


,last_watered,plant_name,soil_moisture,temperature,hours_since_watered,time_since_watered_formatted
0,2025-09-25 13:51:41,Venus flytrap,32.97,14,24.356383,"1 days, 0 hours, 21 minutes"
1,2025-09-25 14:58:34,Corpse flower,34.15,14,23.241661,"0 days, 23 hours, 14 minutes"
2,2025-09-25 13:58:19,Rafflesia arnoldii,35.61,80,24.245828,"1 days, 0 hours, 14 minutes"
3,2025-09-25 14:56:07,Black bat flower,36.58,16,23.282494,"0 days, 23 hours, 16 minutes"
4,2025-09-25 13:57:08,Pitcher plant,29.81,16,24.265550,"1 days, 0 hours, 15 minutes"
...,...,...,...,...,...,...
10358,2025-09-26 13:08:03,Medinilla Magnifica,99.76,95,1.083606,"0 days, 1 hours, 5 minutes"
10359,2025-09-25 14:50:57,Calliandra Haematocephala,-14.96,12,23.368606,"0 days, 23 hours, 22 minutes"
10360,2025-09-25 14:09:29,Zamioculcas Zamiifolia,18.43,15,24.059717,"1 days, 0 hours, 3 minutes"
10361,2025-09-25 14:08:03,Crassula Ovata,20.23,17,24.083606,"1 days, 0 hours, 5 minutes"


In [270]:
last_watered_df.loc[last_watered_df['soil_moisture'] < 0, 'soil_moisture'] = None

In [271]:

alt.Chart(last_watered_df, title='Time since each plant was watered').mark_bar().encode(
    y=alt.Y('plant_name', title='Plant Name',
            axis=alt.Axis(labelLimit=200), sort='-x'),
    x=alt.X('hours_since_watered', title='Hours since watered'),
    tooltip=[alt.Tooltip('time_since_watered_formatted', title='Time since plant was watered'),
             alt.Tooltip('soil_moisture', title='Soil moisture'),
             alt.Tooltip('temperature', title='Temperature')]
)

alt.Chart(...)